In [ ]:
import sys
import yaml
import subprocess

In [ ]:
from pathlib import Path

# Compile original Lyra2 implementations

Need to import python functions that compile Lyra from a git submodule. Manipulate path as per https://stackoverflow.com/a/29747054/1269892

In [ ]:
sys.path.append(str(Path('../lyra/Lyra2/tests').resolve()))

In [ ]:
from harness import build_lyra2

In [ ]:
with open('harness.yml', 'r') as config:
    params = yaml.load(config)

In [ ]:
build_lyra2(params)

In [ ]:
build_path0 = Path('./bin42').resolve()
if not build_path0.exists():
    print('Could not find ./bin42, directory for original executables')

# Compile ported Java implementation

In [ ]:
subprocess.run(['mvn', 'package', '-f', '../lyra2-java', '-Plyra2-compare', '-DskipTests'])

In [ ]:
build_path1 = Path('./target').resolve()
if not build_path1.exists():
    print('Could not ./target, directory for compiled executables')